In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import sys
import os
from tabulate import tabulate
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')



[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [212]:
def upload_sql():

    file_path = input("Enter the CSV filename to upload to SQL: ")

    if not os.path.isfile(file_path):
        print(f"Error: File '{file_path}' not found. Check for any typos")
        return

    table = os.path.splitext(os.path.basename(file_path))[0]
    #db = 'SQL_Datasets'
    user = 'root'
    password = 'Dsci-551'
    host = 'localhost'
    port = 3306

    connection = pymysql.connect(user = user, password = password, host = host, port = port)
    cursor = connection.cursor()

    cursor.execute("Show Databases")
    databases = [db[0] for db in cursor.fetchall()]

    print("\nAvailable Databases: ")
    for i, db in enumerate(databases, start = 1):
        print(f"{i}, {db}")


    while True:
        db_choice = input("\n Enter the name of database you want to upload the CSV to (or type 'New' to create a new database): ").strip()

        if db_choice in databases:
            db = db_choice
            cursor.execute(f"Use `{db}`")
            break
        elif db_choice.lower() == 'new':
            new_db = input("Enter the name of the new database: ").strip()
            cursor.execute(f"Create database if not exists `{new_db}`")
            cursor.execute(f"Use `{new_db}`")
            db = new_db
            break
        else:
            print("Did not recognize input. Please enter one of the existing databases of 'New'")
            
   

    cursor.execute(f"Show tables like '{table}'")
    table_exists = cursor.fetchone() is not None

    if table_exists:
        while True:
            replace = input(f"The table '{table}' already exists. Do you want to replace it? (yes/no): ").strip().lower()
            if replace == 'yes':
                break
            elif replace == 'no':
                print(f"Table '{table}' not replaced")
                connection.close()
                return
            else:
                print("Invalid input. Please enter 'yes' or 'no' only")

    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}')
    data = pd.read_csv(file_path, encoding = 'ISO-8859-1')
    data.to_sql(name = table, con = engine, if_exists = 'replace', index = False)

    print(f"'{file_path} has been successfully uploaded as '{table}' table in database '{db}'")
    cursor.close()
    connection.close()
    

In [213]:
def connect_to_SQL():
    db = 'SQL_Datasets'
    user = 'root'
    password = 'Dsci-551'
    host = 'localhost'
    port = 3306
    connection = pymysql.connect(user = user, password = password, host = host, port = port, database = db)
    return connection
    

In [214]:
def preprocess(user_input):

    tokens = word_tokenize(user_input, language='english')
    tokens = [word.lower() for word in tokens if word.isalnum()]
    stop_words = set(stopwords.words('english'))
    fixed_stop_words = stop_words - {'where', 'having'}

    lemmatizer = WordNetLemmatizer()
    tagged_tokens = pos_tag(tokens)

    def get_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        if tag.startswith('V'):
            return wordnet.VERB
        if tag.startswith('N'):
            return wordnet.NOUN
        if tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    filtered_tokens = [lemmatizer.lemmatize(word, get_pos(tag))
                       for word, tag in tagged_tokens
                       if word not in fixed_stop_words]
    return filtered_tokens, tagged_tokens

In [215]:
def preprocess(user_input):

    tokens = word_tokenize(user_input, language='english')
    tokens = [word.lower() for word in tokens if word.isalnum()]
    stop_words = set(stopwords.words('english'))
    fixed_stop_words = stop_words - {'where', 'having'}
    filtered_tokens = [word for word in tokens if word not in fixed_stop_words]
    tagged_tokens = pos_tag(filtered_tokens)

    return filtered_tokens, tagged_tokens

In [216]:
def column_types(cursor, table):
    cursor.execute(f"Show Columns From `{table}`")
    
    columns_type = cursor.fetchall()
    columns = [col[0] for col in columns_type]
    numeric_columns = [col[0] for col in columns_type if col[1] in ['int', 'float', 'double', 'decimal', 'smallint', 'tinyint', 'bigint']]
    categorical_columns = [col[0] for col in columns_type if col[1] in ['varchar', 'char', 'text', 'enum', 'nchar', 'nvarchar', 'ntext']]

    return columns, numeric_columns, categorical_columns

In [217]:
def sql_sample_data():


    connection = connect_to_SQL()
    cursor = connection.cursor()

    cursor.execute("Show Databases")
    databases = [db[0] for db in cursor.fetchall()]

    print("\nDatabases: ")
    for i, db in enumerate(databases, start = 1):
        print(f"{i}. {db}")

  

    while True:
        selected_db = input("\nEnter the name of database you would like to view: ").strip()
        if selected_db in databases:
            break
        print(f"Error: '{selected_db}' not found. Please enter a valid database.")

    cursor.execute(f"Use `{selected_db}`")

    
    cursor.execute("Show Tables")
    tables = cursor.fetchall()

    if not tables:
        print(f"No table founds in '{db}'. Please upload some.")
        connection.close()
        return
    
    print(f"Tables in '{db}':")

    table_columns = {}
    for i, table in enumerate(tables, start = 1):
        name = table[0]
        cursor.execute(f"Show columns from `{name}`")
        columns = []
        for column in cursor.fetchall():
            columns.append(column[0])

        table_columns[name] = columns
        print(f"\b {name}")
        print(f"Attributes:", ", ".join(columns))
        print()


    table = input("Enter the name of table you want to view sample data from: ")

    if table not in table_columns:
        print(f"Error: {table} does not exist in the database")
        connection.close()
        return


    columns = table_columns.get(table)


    cursor.execute(f"Select * from `{table}` limit 5")
    sample = cursor.fetchall()

    if sample:
        df = pd.DataFrame(sample, columns = columns)
        print(f"\n Sample data from '{table}': ")
        print(tabulate(df, headers = 'keys', tablefmt = 'psql', showindex = False))

    else:
        print(f"No data found in '{table}'")

    connection.close()
    
    

In [218]:
def generate_query_pattern(query, table_choice, columns, numeric_columns, categorical_columns, cursor, num_queries, filtered_tokens):
    
    conditions = ['>', '<', '=', '<=', '>=']
    condition_text = {'>': 'greater than', '<': 'less than', '=': 'equal to', '<=': 'less than or equal to', '>=': 'greater than or equal to'}
    agg_functions = ['AVG', 'SUM', 'MAX', 'MIN']
    agg_text = {'AVG': 'Average', 'SUM': 'Sum', 'MAX': 'Maximum', 'MIN': 'Minimum'}

    def condition_value(cursor, agg_col, table_choice): 
        condition = random.choice(conditions)
        cursor.execute(f"Select MIN(`{agg_col}`), MAX(`{agg_col}`) from `{table_choice}`")
        
        min_value, max_value = cursor.fetchone()
        
        if min_value is None or max_value is None:
            print(f"Error: Value was None")
            return condition, 0
            
        value = random.uniform(min_value, max_value)

        if min_value == int(min_value) and max_value == int(max_value):
            value = round(value)

        return condition, value

    sample_queries = []

    for i in range(num_queries):
        
        if query == 'select':
            columns_used = ', '.join(random.sample(columns, k = min(4, len(columns))))
            query = f"Select `{columns_used}` from `{table_choice}`;"
            nl = f"Select Attributes from {table_choice}"
            sample_queries.append((f"NL Description: {nl}", f"Query: {query}"))

        # where
        elif query == 'where':
            columns_used = ', '.join(random.sample(columns, k = min(4, len(columns))))
            agg_col = random.choice(numeric_columns)
            condition, value = condition_value(cursor, agg_col, table_choice)
            query = f"Select `{columns_used}` from `{table_choice}` where `{agg_col}` {condition} {value};"
            nl = f"Select Attributes from {table_choice} where {agg_col} is {condition_text[condition]} {value}"
            sample_queries.append((f"NL Description: {nl}", f"Query: {query}"))

        # group by
        elif any(token in filtered_tokens for token in ['group'] + agg_functions) and numeric_columns and categorical_columns:

            group = 'group' in filtered_tokens
            agg_function = next((token.upper() for token in filtered_tokens if token.upper() in agg_functions), random.choice(agg_functions))
                
            agg_col = random.choice(numeric_columns)
            group_col = random.choice(categorical_columns)
                
            query = f"Select `{group_col}`, {agg_function}(`{agg_col}`) as agg_`{agg_col}` from `{table_choice}` group by `{group_col}`;"
            nl = f"{agg_text[agg_function]} {agg_col} grouped by {group_col}"
            sample_queries.append((f"NL Description: {nl}", f"Query: {query}"))

                
        elif query == 'count':
                group_col = random.choice(categorical_columns)
                agg_col = random.choice(numeric_columns+categorical_columns)

                if agg_col in numeric_columns:
                    condition, value = condition_value(cursor, agg_col, table_choice)
                    query = f"Select COUNT(*) as count_`{agg_col}` from `{table_choice}` where `{agg_col}` {condition} {value} group by `{group_col}`;"
                    nl = f"Count of {agg_col} in {table_choice} grouped by {group_col} where {agg_col} is {condition_text[condition]} {value}"

                elif agg_col in categorical_columns:
                    query = f"Select `{group_col}`, count(`{agg_col}`) as count_`{agg_col}` from `{table_choice}` group by `{group_col}`"
                    nl = f"Count of {agg_col} in {table_choice} grouped by {group_col}"   
                sample_queries.append((f"NL Description: {nl}", f"Query: {query}"))

        # order by    
        elif query == 'order' and numeric_columns:
            columns_used = ', '.join(random.sample(columns, k = min(4, len(columns))))
            order_col = random.choice(numeric_columns)
            value = random.randint(1, 5)
            query = f"Select `{columns_used}` from `{table_choice}` order by `{order_col}` DESC LIMIT {value};"
            nl = f"Top {value} records order by {order_col}"
            sample_queries.append((f"NL Description: {nl}", f"Query: {query}"))

        # having
        elif query == 'having' and numeric_columns and categorical_columns:
            agg_col = random.choice(numeric_columns)
            group_col = random.choice(categorical_columns)
            agg_function = random.choice(agg_functions)
            condition, value = condition_value(cursor, agg_col, table_choice)
            query = f"Select `{group_col}`, {agg_function}(`{agg_col}`) as `{agg_text[agg_function]}_{agg_col}` from `{table_choice}` group by `{group_col}` having {agg_function}(`{agg_col}`) {condition} {value};"
            nl = f"{agg_text[agg_function]} {agg_col} in {table_choice} grouped by {group_col} given that {agg_text[agg_function]}_{agg_col} {condition_text[condition]} {value}"
            sample_queries.append((f"NL Description: {nl}", f"Query: {query}"))

        elif query == 'nested' and numeric_columns and categorical_columns:
            inner_col = random.choice(numeric_columns)
            outer_col = random.choice(categorical_columns)
            agg_function = random.choice(agg_functions)
            condition, value = condition_value(cursor, inner_col, table_choice)
            limit_num = random.randint(1, 5)
        
            query = (f"Select `{outer_col}`, (select {agg_function}(`{inner_col}`)from `{table_choice}` "
                     f"where `{inner_col}` {condition} {value}) as agg_value "
                     f"from `{table_choice}` "
                     f"where `{outer_col}` is NOT NULL "
                     f"order by agg_value DESC "
                     f"limit {limit_num};")

            nl = f"Nested query with {agg_text[agg_function]} of {inner_col} filtered by {outer_col} where {inner_col} {condition_text[condition]} {value}"
            
            sample_queries.append((f"NL Description: {nl}", f"Query: {query}"))
            
    return sample_queries

    

In [241]:
def gen_sample_queries(user_input = None, num_queries = 3):


    connection = connect_to_SQL()
    cursor = connection.cursor()

    cursor.execute("Show Tables")
    tables = [table[0] for table in cursor.fetchall()]


    if not tables:
        print("No tables in this database")
        connection.close()
        return []
    
    table_choice = random.choice(tables)

    columns, numeric_columns, categorical_columns = column_types(cursor, table_choice)
      
    queries = ['select', 'where', 'group', 'max', 'min', 'avg', 'sum', 'count', 'order', 'having', 'nested']
   
    filtered_tokens, tagged_tokens = preprocess(user_input)
   

    if 'any' in filtered_tokens or not any(query in filtered_tokens for query in queries):
        #print("Generating any query. You selected 'any' or had no specific query identified.")
        selected_queries = []
        for query in queries:
            selected_queries = random.sample(queries, k = num_queries)
        
    else:
        specific_query = [query for query in queries if query in filtered_tokens]
        selected_queries = random.choices(specific_query, k = num_queries)

    sample_queries = []

    for query in selected_queries:
        generated_queries = generate_query_pattern(query = query, table_choice = table_choice, columns = columns,
                                                 numeric_columns = numeric_columns, categorical_columns = categorical_columns, 
                                                 cursor = cursor, 
                                                 num_queries = 1,
                                                 filtered_tokens = filtered_tokens)
        sample_queries += generated_queries

    connection.close()
    
    if sample_queries: 
        return sample_queries

    else: 
        print("No queries could be generated")
        return []
    

In [242]:
def answer_questions(user_input):

    connection = connect_to_SQL()
    cursor = connection.cursor()
    
    sample_queries = gen_sample_queries(user_input = user_input, num_queries = 1)
    print(sample_queries)

    filtered_tokens, tagged_tokens = preprocess(user_input)
    print(filtered_tokens)
    print(tagged_tokens)

  
    def get_table(user_input, cursor):

        filtered_tokens, tagged_tokens = preprocess(user_input)
        cursor.execute("Show Tables")
        tables = [row[0] for row in cursor.fetchall()]

        for table in tables:
            columns, numeric_columns, categorical_columns = column_types(cursor, table)
            if any(token in columns for token in filtered_tokens):
                return table, columns, numeric_columns, categorical_columns

        return None, None, None, None

    table, columns, numeric_columns, categorical_columns = get_table(user_input, cursor)
    
    print(table)

    if not table:
        print("Unable to determine the table. Please use exact column names with tables in input.")
        return None



  
    return table
    
    

In [243]:
if __name__ == "__main__":

    print("Welcome to ChatDB\n")
    while True:

        print("Please select a numbered option: \n")
        print("1. Upload Data\n")
        print("2. View Table Attributes and Sample Data\n")
        print("3. Generate Sample Queries\n")
        print("4. Answer Natural Language Questions\n")
        print("5. End Program\n")

        choice = input("Enter your choice ('1', '2', '3', '4', 5): ").strip()

        if choice.strip() == '1':
            upload_sql()

        elif choice.strip() == '2':
            sql_sample_data()

        elif choice.strip() == '3':
            user_input = input("What kind of query would you like to see or would you like ChatDB to generate one?").strip().lower()
            queries = gen_sample_queries(user_input, num_queries = 3)
            print("\nSample Queries")
            for i, (nl, query) in enumerate(queries, 1):
                print(f"Sample Query {i}: {query}\n")

        elif choice.strip() == '4':
            user_input = input("Ask a question related to the data?").strip().lower()
            answer_questions(user_input)

        elif choice.strip() == '5':
            break
        

        else:
            print("Invalid choice. Please Enter one of the options in quotes above")
    

Welcome to ChatDB

Please select a numbered option: 

1. Upload Data

2. View Table Attributes and Sample Data

3. Generate Sample Queries

4. Answer Natural Language Questions

5. End Program



Enter your choice ('1', '2', '3', '4', 5):  3
What kind of query would you like to see or would you like ChatDB to generate one? any


NameError: name '_queries' is not defined